In [2]:
from Bio.Seq import Seq
import pandas as pd
import subprocess
import random
import time
import glob
import sys
import re

In [2]:
# pwd

In [4]:
# seq_r1_files = glob.glob('/home/pylesh/designs/repeats/library_analysis/miseq_files/*_R1_*fastq')
# seq_r2_files = glob.glob('/home/pylesh/designs/repeats/library_analysis/miseq_files/*_R2_*fastq')
chipmunk_files = glob.glob('/home/pylesh/designs/repeats/library_analysis/chip_ordered_sequences/*txt')

In [4]:
chipmunk_files.sort()
subpool_definitions = {'chip-162-588000-300-pool-2922':'SP1', 'chip-162-588000-300-pool-2923':'SP1', 'chip-162-588000-300-pool-2924':'SP2', 'chip-162-588000-300-pool-2925':'SP2', 'chip-162-588000-300-pool-2926':'SP3', 'chip-162-588000-300-pool-2927':'SP3', 'chip-162-588000-300-pool-2928':'SP4', 'chip-162-588000-300-pool-2929':'SP4', 'chip-162-588000-300-pool-2930':'SP5', 'chip-162-588000-300-pool-2931':'SP5', 'chip-162-588000-300-pool-2932':'SP6', 'chip-162-588000-300-pool-2933':'SP6', 'chip-162-588000-300-pool-2934':'SP7', 'chip-162-588000-300-pool-2935':'SP7', 'chip-162-588000-300-pool-2936':'SP8', 'chip-162-588000-300-pool-2937':'SP8', 'chip-162-588000-300-pool-2938':'SP9', 'chip-162-588000-300-pool-2939':'SP9', 'chip-162-588000-300-pool-2940':'SP10', 'chip-162-588000-300-pool-2941':'SP10', 'chip-162-588000-300-pool-2942':'SP11', 'chip-162-588000-300-pool-2943':'SP11', 'chip-162-588000-300-pool-2944':'SP12', 'chip-162-588000-300-pool-2945':'SP12', 'chip-162-588000-300-pool-2946':'SP13', 'chip-162-588000-300-pool-2947':'SP13', 'chip-162-588000-300-pool-2948':'SP14', 'chip-162-588000-300-pool-2949':'SP14', 'chip-162-588000-300-pool-2950':'SP15', 'chip-162-588000-300-pool-2951':'SP15', 'chip-162-588000-300-pool-2952':'SP16', 'chip-162-588000-300-pool-2953':'SP16'}

In [5]:
all_sequences = []
seq_sources = []
seq_subpools = []
unique_ids = []

for file in chipmunk_files:
    file_df = pd.read_csv(file, header=None)
    chipmunk_source = file.split('/')[-1].replace('.txt', '')
    for i, seq in enumerate(file_df[0]):
        all_sequences.append(seq)
        seq_sources.append(chipmunk_source)
        SP = subpool_definitions[chipmunk_source]
        seq_subpools.append(SP)
        ID = (str(i)).rjust(5, '0')
        unique_ids.append(f'{SP}_{ID}_{chipmunk_source}')
    
chipmunk_df = pd.DataFrame()
chipmunk_df['dna'] = all_sequences
chipmunk_df['source'] = seq_sources
chipmunk_df['subpool'] = seq_subpools
chipmunk_df['id'] = unique_ids

In [59]:
# #KEEP BUT DONT RERUN UNLESS NEEDED
# with open('chip_ordered_sequences/chipmunk_sequences.fas','w') as chipmunk_fasta:
#     for ID, DNA in zip(chipmunk_df['id'], chipmunk_df['dna']):
#         print (f'>{ID}', file=chipmunk_fasta)
#         print (f'{DNA}', file=chipmunk_fasta)

# for source in chipmunk_df['source']:
#     source_df = chipmunk_df[ chipmunk_df['source']==source ] 
#     with open(f'chip_ordered_sequences/{source}.fas', 'w') as chipmunk_fasta:
#         for ID, DNA in zip(source_df['id'], source_df['dna']):
#             print (f'>{ID}', file=chipmunk_fasta)
#             print (f'{DNA}', file=chipmunk_fasta)



In [60]:
def blast(seq, full=0, short=1, return_seq=0, subject='./identify_enriched_sequences/chip_ordered_sequences/chipmunk_sequences.fas'):
    p1 = subprocess.Popen(['echo', '-e', '>query\n'+seq], stdout=subprocess.PIPE)
    if short:
        p2 = subprocess.Popen(['blastn', '-subject', subject, '-task', 'blastn-short', '-evalue', '0.00001', '-perc_identity', '100' ], stdin=p1.stdout, stdout=subprocess.PIPE)
    else:
        p2 = subprocess.Popen(['blastn', '-subject', subject], stdin=p1.stdout, stdout=subprocess.PIPE)

    blast_output = p2.communicate()[0].decode('UTF-8')
    blast_lines = blast_output.split('\n')
    #print ('blast_lines', '\n'.join(blast_lines))
    if full:
        print (blast_output)
    
    for i, line in enumerate(blast_lines):
        if line.startswith('>'):
            name = line.split()[-1]
            identity_line = blast_lines[i+4]
            #print ('identity_line', identity_line)
            identity_ratio = re.sub(r'^ Identities = (\d+\/\d+).*\n?', r'\1', identity_line)
            #print ('identity_ratio', identity_ratio)
            identity = float(identity_ratio.split('/')[0]) / float(identity_ratio.split('/')[1])
            length = int(identity_ratio.split('/')[1])
            if not return_seq:
                yield (identity, length, name)
            else:
                query = blast_lines[i+7].split()[2]
                yield (identity, length, name, query.upper() )
            


In [51]:
cterm_seqs = ['chip-162-588000-300-pool-2923', 'chip-162-588000-300-pool-2925', 'chip-162-588000-300-pool-2927', 'chip-162-588000-300-pool-2929', 'chip-162-588000-300-pool-2931', 'chip-162-588000-300-pool-2933', 'chip-162-588000-300-pool-2935', 'chip-162-588000-300-pool-2937', 'chip-162-588000-300-pool-2939', 'chip-162-588000-300-pool-2941', 'chip-162-588000-300-pool-2943', 'chip-162-588000-300-pool-2945', 'chip-162-588000-300-pool-2947', 'chip-162-588000-300-pool-2949', 'chip-162-588000-300-pool-2951', 'chip-162-588000-300-pool-2953']
nterm_seqs = ['chip-162-588000-300-pool-2922', 'chip-162-588000-300-pool-2924', 'chip-162-588000-300-pool-2926', 'chip-162-588000-300-pool-2928', 'chip-162-588000-300-pool-2930', 'chip-162-588000-300-pool-2932', 'chip-162-588000-300-pool-2934', 'chip-162-588000-300-pool-2936', 'chip-162-588000-300-pool-2938', 'chip-162-588000-300-pool-2940', 'chip-162-588000-300-pool-2942', 'chip-162-588000-300-pool-2944', 'chip-162-588000-300-pool-2946', 'chip-162-588000-300-pool-2948', 'chip-162-588000-300-pool-2950', 'chip-162-588000-300-pool-2952']

design_list = []
status_list = []
subpool_list = []
aa_list = []
dna_list = []
fiveprime_list = []
threeprime_list = []

for cterm, nterm in zip(cterm_seqs, nterm_seqs):
    cterm_df = chipmunk_df[ chipmunk_df['source']==cterm ]    
    nterm_df = chipmunk_df[ chipmunk_df['source']==nterm ]    
    nterm_hash = {name:seq for name, seq in zip(nterm_df['id'], nterm_df['dna'])}
    
    #print (cterm, nterm,'cterm, nterm')
    sp = subpool_definitions[cterm]

    for cseq, cname in zip(cterm_df['dna'], cterm_df['id']):
        blast_hits = blast(cseq, subject=f'chip_ordered_sequences/{nterm}.fas', return_seq=1) #, full=1)
        first = 1
        status = 'design'
        for one, length, nname, match in blast_hits:
            nseq = nterm_hash[nname]
            if first: 
                max_length = length
                first = 0
            
            assembled_seq = nseq.split(match)[0] + match + cseq.split(match)[-1]
            protein_seq = str(Seq(assembled_seq).translate(to_stop=True))
            design_seq = re.sub(r'^[GS]+SGG.{9,13}K[GS]?[GS]?..(.*?)[GS]*$', r'\1', protein_seq)
            
            try:
                assert design_seq != protein_seq, f'{protein_seq}\n{design_seq}'
                try:
                    designed_grep = subprocess.check_output(['grep', design_seq, '-B', '1', 'chip_ordered_sequences/non_redundant_output.fas'])
                except:
#                     print (f'Error GREPPING protein_seq:\n>design_seq\n{design_seq}\n>{nname}\n{nseq}\n>{cname}\n{cseq}\nstatus:{status}\nmatch:{match}')
                    status = 'chimera'
                    nseq_trunc = nseq.split(match)[0]
                    assert nseq_trunc != nseq, f'\n>{nname}\n{nseq}\n>{cname}\n{cseq}\n>match\n{match}'
                    nseq_trunc = nseq_trunc[:int(len(nseq_trunc)/3)*3]
                    nterm_protein = str(Seq(nseq_trunc).translate(to_stop=True))
                    design_nterm = re.sub(r'^[GS]+SGG.{9,13}K[GS]?[GS]?..(..*?)$', r'\1', nterm_protein)
                    try:
                        nterm_designed_grep = subprocess.check_output(['grep', design_nterm, '-B', '1', 'chip_ordered_sequences/non_redundant_output.fas'])
                    except:
#                         print (f'Nterm Error GREPPING design_nterm:\n>design_nterm\n{design_nterm}\n>nterm_protein\n{nterm_protein}\n>{nname}\n{nseq}\n>{cname}\n{cseq}\nstatus:{status}\nmatch:{match}')
                        nterm_designed_grep = 'junk'
                    
                    # WORK HERE
                    cseq_trunc = cseq.split(match)[-1]
                    cseq_trunc = cseq_trunc[len(cseq_trunc) % 3:]
                    cterm_protein = str(Seq(cseq_trunc).translate(to_stop=True))
                    design_cterm = re.sub(r'^(.+?)[GS]+$', r'\1', cterm_protein)
                    try:
                        cterm_designed_grep = subprocess.check_output(['grep', design_cterm, '-B', '1', 'chip_ordered_sequences/non_redundant_output.fas'])
                    except:
#                         print (f'Cterm Error GREPPING design_cterm:\n>design_cterm\n{design_cterm}\n>cterm_protein\n{cterm_protein}\n>{cname}\n{cseq}\n>{cname}\n{cseq}\nstatus:{status}\nmatch:{match}')
                        cterm_designed_grep = 'junk'
                        
            except:
                status = 'junk'
            
            if status == 'design':
                #print ('DESIGN')
                design = designed_grep.decode('UTF-8').split('\n')[0][1:]
                
            elif status == 'chimera':
                #print ('CHIMERA')

                if nterm_designed_grep != 'junk':
                    nterm_name = nterm_designed_grep.decode('UTF-8').split('\n')[0][1:]
                else:
                    nterm_name = design_nterm
                    #nterm_name = 'junk'
                    
                if cterm_designed_grep != 'junk':
                    cterm_name = cterm_designed_grep.decode('UTF-8').split('\n')[0][1:]                
                else:
                    cterm_name = design_cterm
                    #cterm_name = 'junk' 
                
                design = f'{nterm_name}_CHIMERA_{cterm_name}'
            
            elif status == 'junk':
                #print ('JUNK')
                design = design_seq
            #print (design)
            #print ()

                
            design_list.append(design)
            status_list.append(status)
            subpool_list.append(sp)
            aa_list.append(protein_seq)
            dna_list.append(assembled_seq)
            fiveprime_list.append(nseq)
            threeprime_list.append(cseq)

        
insilico_assembled_df = pd.DataFrame()
insilico_assembled_df['design'] = design_list
insilico_assembled_df['status'] = status_list
insilico_assembled_df['subpool'] = subpool_list
insilico_assembled_df['aa'] = aa_list
insilico_assembled_df['dna'] = dna_list
insilico_assembled_df['5prime'] = fiveprime_list
insilico_assembled_df['3prime'] = threeprime_list

insilico_assembled_df.to_csv('insilico_assembled_sequences.tsv', sep='\t')


In [61]:
insilico_assembled_df = pd.read_csv('insilico_assembled_sequences.tsv', sep='\t')

In [62]:
# Iterate through subpools in (insilico) assembled seqeunces, output fasta files for each subpool 
subpools = [f'SP{str(x)}' for x in range(1,17)]

for pool in subpools:
    subpool_assembled = insilico_assembled_df[insilico_assembled_df['subpool'] == pool]
#     print (subpool_assembled)

    with open(f'chip_ordered_sequences/chip_assembled_{pool}.fas', 'w') as chip_assembled_fasta:
        for design, sequence, status in zip(subpool_assembled['design'], subpool_assembled['dna'], subpool_assembled['status']):
            print (f'>{design}\n{sequence}', file=chip_assembled_fasta)
            